INTAKE

Purpose: The intake is like the ”entry door” for sunlight. It’s designed to gather as much sunlight as possible and
guide it into the Quantum Photonic Array.

Basic Idea: Imagine using a magnifying glass to focus sunlight on a tiny spot; the intake works on a similar
principle. It collects a large amount of sunlight and concentrates or ”focuses” it onto a smaller area, increasing the
energy densit

In [31]:
#   Global variables

import math
import pandas as pd

simulated_day = 1
latitude = 30.27  # Latitude of Austin
longtitude = 97.73 #Longitude of Austin
intensity_at_top_of_atmosphere = 1361  # W/m^2
concentration_factor = 100  # How much sunlight is being concentrated into funnel

In [32]:
#   SOLAR DECLINATION ANGLE
#   The solar declination angle (δ) varies throughout the year. It can be estimated using the day of the year d

#   Where day_of_year = 1 represents January 1st.
def solar_declination(day_of_year):
    return 23.45 * math.sin(math.radians(360 / 365 * (day_of_year + 284)))

In [33]:
#   SOLAR ELEVATION ANGLE
#   The solar elevation angle (α) at solar noon can be determined using the solar declination and Austin’s latitude
#   Where day_of_year = 1 represents January 1st.

def solar_elevation_angle(day_of_year, hour):
    
    delta = solar_declination(day_of_year)
    LST = hour + 4*(90 - longtitude)/60
    H = 15 * (LST - 12)
    elevationAngle = math.degrees(math.asin(math.sin(delta) * math.sin(latitude) + math.cos(delta) * math.cos(latitude) * math.cos(H)))

    return elevationAngle

In [34]:
#   SUNLIGHT INTENSITY
#   The intensity of sunlight (I0(t)) on a clear day outside the Earth’s atmosphere is approximately 1361 W/m2
#   This intensity varies based on the angle of the sun and can be modeled using the cosine of the zenith angle:

#   Where day_of_year = 1 represents January 1st.
def sunlight_intensity(day_of_year, hour):
    alpha = solar_elevation_angle(day_of_year, hour)
    return intensity_at_top_of_atmosphere * math.cos(math.radians(90 - alpha))

In [35]:
#   Main Function
#   Run simulation and save it into pandas dataframe that will be put into excel file

results = []
for day in range(0, 7, 1):  # Simulate each day

    for hour in range(0, 24, 1):  # Simulate each hour

        declination_angle = solar_declination(day)
        elevation_angle = solar_elevation_angle(day, hour)
        intensity = sunlight_intensity(day, hour)
        concentrated_value = concentration_factor * intensity
        

        results.append([day, hour, declination_angle, elevation_angle, intensity, concentration_factor, concentrated_value])

df = pd.DataFrame(results, columns=["Simulated Day", "Hour of Day", "Solar Declination Angle", "Solar Elevation Angle (degrees)", "Sunlight Intensity (W/m^2)", "Concentration Factor", "Intensity * Concentration factor (funnel)"])
df.to_excel("solar_simulation_results.xlsx", index=False)